# YAHOO電影爬蟲練習
## 練習爬取電影放映資訊。必須逐步獲取電影的代號、放映地區、放映日期後，再送出查詢給伺服器。

In [59]:
import requests
import re
from bs4 import BeautifulSoup

### 先搜尋全部的電影代號(ID)資訊

In [60]:
# 查看目前上映那些電影，並擷取出其ID資訊
url = 'https://movies.yahoo.com.tw/'
resp = requests.get(url)
resp.encoding = 'utf-8'

soup = BeautifulSoup(resp.text, 'lxml')
html = soup.find("select", attrs={'name':'movie_id'})
movie_item = html.find_all("option", attrs={'data-name':re.compile('.*')})

for p in movie_item:
    print("Movie: %s, ID: %s" % (p["data-name"], p["value"]))

Movie: 復仇母親, ID: 10392
Movie: 風暴過後, ID: 10387
Movie: 南方車站的聚會, ID: 10380
Movie: 祭旗, ID: 10373
Movie: 唯你是愛, ID: 10370
Movie: 屍控療法, ID: 10367
Movie: 黑熊來了, ID: 10357
Movie: 唐人街探案2, ID: 10353
Movie: 黑計畫, ID: 10351
Movie: HUMAN LOST人間失格, ID: 10348
Movie: 假面騎士劇場版ZI-O Over Quartzer, ID: 10331
Movie: 巴黎小情聖, ID: 10329
Movie: 每個月來訪一次的月經醬, ID: 10324
Movie: 8級警戒, ID: 10323
Movie: 為你存在的每一天, ID: 10297
Movie: 黑色聖誕節, ID: 10291
Movie: 熱氣球飛行家, ID: 10257
Movie: 大說謊家, ID: 10122
Movie: 特約經紀公司, ID: 10364
Movie: 一屍到底番外篇：好萊塢大作戰！, ID: 10352
Movie: 花生醬獵鷹的願望, ID: 10340
Movie: 女孩要革命, ID: 10339
Movie: 我的靈魂是愛做的, ID: 10336
Movie: 絕世情歌, ID: 10328
Movie: 人間失格：太宰治與他的3個女人, ID: 10287
Movie: 抓狂博物館, ID: 10279
Movie: 呆萌特務, ID: 10264
Movie: 去年聖誕節, ID: 10256
Movie: 野蠻遊戲：全面晉級, ID: 10134
Movie: 夕霧花園, ID: 10327
Movie: 懸案密碼前傳：瓶中信, ID: 10254
Movie: 布魯克林孤兒, ID: 10224
Movie: 暴走曼哈頓, ID: 10044
Movie: 鋒迴路轉, ID: 10258
Movie: 賽道狂人, ID: 10097
Movie: 麂皮：永不滿足, ID: 10358
Movie: 82年生的金智英, ID: 10298
Movie: 冰雪奇緣2, ID: 9597
Movie: 我的情敵是冰箱, ID: 10

### 指定你有興趣的電影其ID，然後查詢其放映地區資訊。

In [61]:
# 參考前一個步驟中擷取到的ID資訊，並指定ID
movie_id = 10387

In [62]:
url = 'https://movies.yahoo.com.tw/api/v1/areas_by_movie_theater'
payload = {'movie_id':str(movie_id)}

# 模擬一個header
headers = {
    'authority': 'movies.yahoo.com.tw',
    'method': 'GET',
    'path': '/api/v1/areas_by_movie_theater?movie_id=' + str(movie_id),
    'scheme': 'https',
    'accept': 'application/json, text/javascript, */*; q=0.01',
    'accept-encoding': 'gzip, deflate, br',
    'accept-language': 'zh-TW,zh;q=0.9,en-US;q=0.8,en;q=0.7,zh-CN;q=0.6',
    'cookie': 'rxx=9s3x2fws06.1g16irnc&v=1; _ga=GA1.3.2056742944.1551651301; GUC=AQEBAQFczFpdm0IfmwSB&s=AQAAACoo4N5D&g=XMsVBw; BX=4hkdk1decm57t&b=3&s=mr; _ga=GA1.4.2056742944.1551651301; nexagesuid=82843256dd234e8e91aa73f2062f8218; browsed_movie=eyJpdiI6IlJXWWtiSWJaZlNGK2MxQnhscnVUYWc9PSIsInZhbHVlIjoiMXRhMmVHRXRIeUNjc1RBWDJzdGYwbnlIQURmWGsrcjJSMzhkbkcraDNJVUNIZEZsbzU3amlFcVZ1NzlmazJrTGpoMjVrbHk1YmpoRENXaHZTOUw1TmI2ZTZVWHdOejZQZm16RmVuMWlHTTJLaTZLVFZZVkFOMDlTd1wvSGltcytJIiwibWFjIjoiZWQ2ZjA4MmVjZmZlYjlmNjJmYmY2NGMyMDI0Njc0NWViYjVkOWE2NDg0N2RhODMxZjBjZDhiMmJhZTc2MDZhYiJ9; avi=eyJpdiI6Im1NeWFJRlVRWDR1endEcGRGUGJUbVE9PSIsInZhbHVlIjoickRpU3JuUytmcGl6cjF5OW0rNU9iZz09IiwibWFjIjoiY2VmY2NkNzZmM2NhNjY5YzlkOTcyNjE5OGEyMzU0NWYxOTdmMDRkMDY3OWNmMmZjOTMxYjc5MjI5N2Q5NGE5MiJ9; cmp=t=1559391030&j=0; _gid=GA1.4.779543841.1559391031; XSRF-TOKEN=eyJpdiI6IkhpS2hGcDRQaHlmWUJmaHdSS2Q2bHc9PSIsInZhbHVlIjoiOUVoNFk4OHI1UUZmUWRtYXhza0MyWjJSTlhlZ3RnT0VGeVJPN2JuczVRMGRFdWt2OUlsamVKeHRobFwvcHBGM0dhU3VyMXNGTHlsb2dVM2l0U1hpUGxBPT0iLCJtYWMiOiJkZWU4YzJhNjAxMTY3MzE4Y2ExNWIxYmE1ZjE1YWZlZTlhOTcyYjc4M2RlZGY4ZWNjZDYyMTA2NGYwZGViMzc2In0%3D; m_s=eyJpdiI6InpsZHZ2Tk1BZ0dxaHhETml1RjBnUXc9PSIsInZhbHVlIjoiSkNGeHUranRoXC85bDFiaDhySTJqNkJRcWdjWUxjeVRJSHVYZ1wvd2d4bWJZUTUrSHVDM0lUcW5KNHdETFZ4T1lieU81OUhzc1VoUXhZcWk0UDZSQXVFdz09IiwibWFjIjoiYmJkMDJkMDhlODIzMzcyMWY4M2NmYWNjNGVlOWRjMDIwZmVmNzAyMjE3Yzg3ZGY3ODBkZWEzZTI4MTI5ZWNmOSJ9; _gat=1; nexagesd=10',
    'dnt': '1',
    'mv-authorization': '21835b082e15b91a69b3851eec7b31b82ce82afb',
    'referer': 'https://movies.yahoo.com.tw/',
    'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/74.0.3729.169 Safari/537.36',
    'x-requested-with': 'XMLHttpRequest',}
resp = requests.get(url, params=payload, headers=headers)
# 這裡回傳的格式是JSON格式的資料，要解析JSON擷取資料
#print(resp.json())  # 若有需要，列印出json原始碼

#print(resp.url)
for p in resp.json():
    print('放映地區: {0}, 代號(area_id): {1}'.format(p['title'], p['area_id']))

放映地區: 台北市, 代號(area_id): 28
放映地區: 新北市, 代號(area_id): 8
放映地區: 桃園, 代號(area_id): 16
放映地區: 台南, 代號(area_id): 10
放映地區: 高雄, 代號(area_id): 17


### 指定你想要觀看的放映地區，查詢有上映電影的場次日期

In [63]:
# 指定放映地區
area_id = 28

In [64]:
# 向網站發送請求
url = 'https://movies.yahoo.com.tw/movietime_result.html'
payload = {'movie_id':str(movie_id), 'area_id':str(area_id)}
resp = requests.get(url, params=payload)
resp.encoding = 'utf-8'

soup = BeautifulSoup(resp.text, 'lxml')
movie_date = soup.find_all("label", attrs={'for':re.compile("date_[\d]")})

# 列印播放日期
for date in movie_date:
    print("%s %s" % (date.p.string, date.h3.string))

十二月 13
十二月 14
十二月 15
十二月 16
十二月 17


### 最後指定觀看的日期，查詢並列印出放映的電影院、放映類型(數位、3D、IMAX 3D...)、放映時間等資訊。

In [68]:
# 選定要觀看的日期
date = "2019-12-15"

In [69]:

# 向網站發送請求，獲取上映的電影院及時間資訊
url = "https://movies.yahoo.com.tw/ajax/pc/get_schedule_by_movie"
payload = {'movie_id':str(movie_id),
           'date':date,
           'area_id':str(area_id),
           'theater_id':'',
           'datetime':'',
           'movie_type_id':''}

resp = requests.get(url, params=payload)

# print(resp.json()['view'])  # 若有需要，列印出json原始碼

soup = BeautifulSoup(resp.json()['view'], 'lxml')
html = soup.find_all("ul", attrs={'data-theater_name':re.compile(".*")})


[<ul class="area_time _c jq_area_time" data-theater_name="台北真善美劇院" data-theater_schedules="https://movies.yahoo.com.tw/theater_result.html/id=41" data-theater_url="" id="theater_id_41">
<li class="adds">
<a href="https://movies.yahoo.com.tw/theater_result.html/id=41">台北真善美劇院</a>
<span>02-23312270</span>
</li>
<li class="taps">
<span class="tapR">數位</span> </li>
<li class="time _c">
<div class="input_picker jq_input_picker">
<input class="gabtn" data-cinema="f644412efbb811e58858f2128151146f" data-date="1576339200" data-ezding="ezding" data-ga="['電影頁_時刻表', '電影頁_時刻表_場次', '風暴過後']" data-movie="e6e354a5cf07477b95c45a44ecde43ab" data-movie_date="12.15" data-movie_time="13:25" data-movie_title="風暴過後" data-movie_type="數位" data-session="08f36c8cfe4a42b7904f354e9a127659" data-time="1576387500" id="72561846" name="schedule_list" type="radio" value="2019-12-15 13:25:00"/>
<label class="select" for="72561846">13:25</label>
<input class="gabtn" data-cinema="f644412efbb811e58858f2128151146f" data-date

In [88]:
'''

  試著從上一步驟回傳的電影院資料中，擷取電影院名稱、影片放映類型以及時間表
  
  Your code here.

'''
for h in html:
    print('--------------------------------------------------------------------')
    print('電影院：{}'.format(h['data-theater_name']))
    print('影片放映類型：{}'.format(h.find('span',class_='tapR').text))
    #print('時間表：{}'.format([d['value'] for d in h.find_all('input', class_='gabtn')])) method-1
    print('時間表：{}'.format([d.text for d in h.find_all('label', class_='select')])) #method-2

--------------------------------------------------------------------
電影院：台北真善美劇院
影片放映類型：數位
時間表：['13:25', '19:45']
--------------------------------------------------------------------
電影院：喜樂時代影城南港店
影片放映類型：數位
時間表：['10:00', '14:50', '19:00']
